In [73]:
from PyPDF2 import PdfReader
import os
import sys
import openai
import sqlite3
import pickle

In [74]:
dir = '/Users/katiegreed/Documents/Duke AI Masters/AIPI 590 - Large Language Models/HW/Assignment2/Docs/'

data = []
data_names = []
for doc in os.listdir(dir):
    print(doc)
    try:
        reader = PdfReader(dir + doc)
        content = ''
        for page in reader.pages:
            content += page.extract_text()
        data.append(content)
        data_names.append(doc)
    except:
        print(f"Could not read {doc}. Skipping...")

LEADERSHIP OF THE COXSWAIN .pdf
.DS_Store
Could not read .DS_Store. Skipping...
Rules_Of_Rowing_2023_FINAL_PDF.pdf
Coaching-the-Coxswain-EXTRACT.pdf
Could not read Coaching-the-Coxswain-EXTRACT.pdf. Skipping...
TCC Coxswain Handbook 4.pdf
2022-23D1WCR_PreChampionshipManual.pdf
FiveKeystoCompetitiveCoxing.pdf
Coxswain Commands.pdf


In [75]:
data[0]

' \nContact Information:  Yaz Farooq , Head Coach Women\'s Rowing  \nStanford University,  Tel: (650) 387-8969 (cell)  \nyfarooq@stanford.edu      www.gostanford.com       \nwww.COXSWAINation.com  LEADERSHIP OF THE COXSWAIN \n \nYasmin Farooq \n \nGoal:  To develop leaders who will implement your coaching philosophies and strategies \nin practice and on race day.  Discussion will explore the difference between coxswain and rower leaders, making the most of both, a nd how they can work together to create a \ndynamic and productive environment for the team. \nOutline: \n \n1. Determining the actual ‘roles’ of coxswain and rower leaders \n \n2. Walking the Tightrope: Day to Day Leadership \nThe cox may direct the crew in practices and races, but s/he is not performing the \nphysical tasks.  How the cox can support the coach and team and still maintain the \nposition of “teammate.” \n \n3. Determining how much responsibility to  assign the coxswain in practice \n \n4. The importance of “Qu

In [76]:
data_names

['LEADERSHIP OF THE COXSWAIN .pdf',
 'Rules_Of_Rowing_2023_FINAL_PDF.pdf',
 'TCC Coxswain Handbook 4.pdf',
 '2022-23D1WCR_PreChampionshipManual.pdf',
 'FiveKeystoCompetitiveCoxing.pdf',
 'Coxswain Commands.pdf']

In [87]:
'''
Set up SQlite database to store the chunks and their corresponding document names
'''
def create_db():
    conn = sqlite3.connect('coxswain_rag.db')
    c = conn.cursor()
    
    # Drop the 'chunks' table if it exists
    c.execute("DROP TABLE IF EXISTS chunks")

    c.execute('''CREATE TABLE IF NOT EXISTS chunks (
              id INTEGER PRIMARY KEY, doc_name TEXT, chunk TEXT, embeddings BLOB, cosine_simiarlity FLOAT
              )''')
    conn.commit()
    conn.close()

create_db()

In [99]:
'''Create chunks of text that can be processed and vectorized.
Want the chunks to end at the end of a senetence and the end of a paragraph.

Parameters: 
text - a string to be chunked 
Returns: a list of chunks of text
'''
def create_chunks(text, doc_name, chunk_size=500):
    # Initialize variables
    chunks = []
    current_chunk = ''
    current_size = 0
    
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')

    # Iterate through each paragraph
    for paragraph in paragraphs:
        # Split the paragraph into sentences
        sentences = paragraph.split('. ')
        
        # Iterate through each sentence
        for sentence in sentences:
            # Check if adding the sentence to the current chunk would exceed the chunk size
            if len(current_chunk) + len(sentence) + 2 <= chunk_size:  # Add 2 for the period and space
                # Add the sentence to the current chunk
                if current_chunk:
                    current_chunk += '. ' + sentence
                else:
                    current_chunk += sentence
                current_size += len(sentence) + 2  # Update the current size
            else:
                # Add the current chunk to the list of chunks
                chunks.append(current_chunk)
                # Reset the current chunk and size to start a new chunk
                current_chunk = sentence
                current_size = len(sentence)
        
        # Add the last chunk of the paragraph
        if current_chunk:
            chunks.append(current_chunk)
            current_chunk = ''
            current_size = 0
    
    # Return the list of chunks
    print(f'Created {len(chunks)} chunks')

    # Add the chunks to the database
    conn = sqlite3.connect('coxswain_rag.db')
    c = conn.cursor()
    for chunk in chunks:
        c.execute('INSERT INTO chunks (doc_name, chunk) VALUES (?, ?)', (doc_name, chunk))
    conn.commit()
    conn.close()

    return chunks

chunks = create_chunks(data[1], data_names[1])

Created 732 chunks


In [100]:
#View the database
conn = sqlite3.connect('coxswain_rag.db')
c = conn.cursor()
c.execute('SELECT * FROM chunks')
rows = c.fetchall()
for row in rows:
    print(row)
conn.close()

(1, 'LEADERSHIP OF THE COXSWAIN .pdf', " \nContact Information:  Yaz Farooq , Head Coach Women's Rowing  \nStanford University,  Tel: (650) 387-8969 (cell)  \nyfarooq@stanford.edu      www.gostanford.com       \nwww.COXSWAINation.com  LEADERSHIP OF THE COXSWAIN \n \nYasmin Farooq \n \nGoal:  To develop leaders who will implement your coaching philosophies and strategies \nin practice and on race day", b'\x80\x04\x95\x076\x00\x00\x00\x00\x00\x00]\x94(G?\x82Qo`\x00\x00\x00G\xbfi\xa3\xe9`\x00\x00\x00G?\xa4\x92\xb2@\x00\x00\x00G?\x9f\xdf|\xc0\x00\x00\x00G\xbf}\x02\x8f\x80\x00\x00\x00G\xbf\x9asw`\x00\x00\x00G\xbft\xd0P`\x00\x00\x00G\xbf\x91A\x05\x00\x00\x00\x00G\xbf\x82[*\x00\x00\x00\x00G\xbf\xa8\xa3\xb6@\x00\x00\x00G?\x9a,\x1e\xa0\x00\x00\x00G\xbf\x99\xb4 \x80\x00\x00\x00G\xbf\x84\xc3W\x80\x00\x00\x00G\xbf\x9bC\x05`\x00\x00\x00G?\xa3\nI\xc0\x00\x00\x00G?\x8c9}\xe0\x00\x00\x00G?\x7f\xf9n\x80\x00\x00\x00G\xbf\x92\xe9\xdb\xa0\x00\x00\x00G?\x9bO\xfe@\x00\x00\x00G?\xa9v\x82`\x00\x00\x00G?\x88\x

In [101]:
'''
Creates embeddings for the chuncks of text using OpenAI's text embedding model. 
Connects to the SQLite database and stores the embeddings in a table.

Parameters:
chunks - a list of strings to be embedded
table_name - the name of the table to store the embeddings

Returns: None
'''
def create_embeddings(chunks, model="text-embedding-3-small"):

    # Connect to the database
    conn = sqlite3.connect('coxswain_rag.db')
    c = conn.cursor()

    # Create embeddings for each chunk
    client = openai.Client()
    embeddings = []

    for chunk in chunks:
        result = (client.embeddings.create(input=chunk, model=model)).data[0].embedding
        result_pickle = pickle.dumps(result)
        
        # Store the embeddings in the database
        c.execute("UPDATE chunks SET embeddings = ? WHERE chunk = ?", (sqlite3.Binary(result_pickle), chunk))
        conn.commit()
    
    # Close the database connection
    conn.close()

create_embeddings(chunks)

In [91]:
#View the database
conn = sqlite3.connect('coxswain_rag.db')
c = conn.cursor()
c.execute('SELECT * FROM chunks')
rows = c.fetchall()
for row in rows:
    print(row)
conn.close()

(1, 'LEADERSHIP OF THE COXSWAIN .pdf', " \nContact Information:  Yaz Farooq , Head Coach Women's Rowing  \nStanford University,  Tel: (650) 387-8969 (cell)  \nyfarooq@stanford.edu      www.gostanford.com       \nwww.COXSWAINation.com  LEADERSHIP OF THE COXSWAIN \n \nYasmin Farooq \n \nGoal:  To develop leaders who will implement your coaching philosophies and strategies \nin practice and on race day", b'\x80\x04\x95\x076\x00\x00\x00\x00\x00\x00]\x94(G?\x82Qo`\x00\x00\x00G\xbfi\xa3\xe9`\x00\x00\x00G?\xa4\x92\xb2@\x00\x00\x00G?\x9f\xdf|\xc0\x00\x00\x00G\xbf}\x02\x8f\x80\x00\x00\x00G\xbf\x9asw`\x00\x00\x00G\xbft\xd0P`\x00\x00\x00G\xbf\x91A\x05\x00\x00\x00\x00G\xbf\x82[*\x00\x00\x00\x00G\xbf\xa8\xa3\xb6@\x00\x00\x00G?\x9a,\x1e\xa0\x00\x00\x00G\xbf\x99\xb4 \x80\x00\x00\x00G\xbf\x84\xc3W\x80\x00\x00\x00G\xbf\x9bC\x05`\x00\x00\x00G?\xa3\nI\xc0\x00\x00\x00G?\x8c9}\xe0\x00\x00\x00G?\x7f\xf9n\x80\x00\x00\x00G\xbf\x92\xe9\xdb\xa0\x00\x00\x00G?\x9bO\xfe@\x00\x00\x00G?\xa9v\x82`\x00\x00\x00G?\x88\x

In [102]:
def embed_query(query, model="text-embedding-3-small"):
    # Create an embedding for the query
    client = openai.Client()
    result = client.embeddings.create(input=query, model=model)
    return result.data[0].embedding

In [112]:
'''
Semantic search the embeddings using cosine similarity to locate the most similar chunk given a query.
'''
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import sqlite3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_and_store_cosine_similarity(query_embedding):
    # Connect to the SQLite database
    conn = sqlite3.connect('coxswain_rag.db')
    cursor = conn.cursor()

    # Fetch all embeddings from the database
    cursor.execute("SELECT chunk, embeddings FROM chunks WHERE embeddings IS NOT NULL")
    rows = cursor.fetchall()

    # Calculate cosine similarity for each embedding
    similarities = []
    for row in rows:
        chunk, embedding_pickle = row
        # Convert binary data back to numpy array
        embedding = pickle.loads(embedding_pickle)
        # Calculate cosine similarity
        similarity_score = cosine_similarity([query_embedding], [embedding])[0][0]
        similarities.append((chunk, similarity_score))

    # Store the similarity scores in the database
    for chunk, similarity_score in similarities:
        cursor.execute("UPDATE chunks SET cosine_simiarlity = ? WHERE chunk = ?", (similarity_score, chunk))
    conn.commit()

    # Get the top 3 similarity scores
    top_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:5]

    # Close cursor and connection
    cursor.close()
    conn.close()

    return top_similarities

# Example usage
query = "How does a quick start work?"
top_similarity_scores = calculate_and_store_cosine_similarity(embed_query(query))
for chunk, similarity_score in top_similarity_scores:
    print("Chunk:", chunk)

    # Print the similarity score
    print("Similarity score:", similarity_score)
    print('--------\n')


Chunk:   
2-307 “Quick Start”  
If weather or other conditions prevent use of the normal starting procedure, the Starter may dis pense with 
the announcement of C rews described in Rule 2 -305 (“Start”) . The Start er first shall announce to the 
Crews that a “ Quick Start” will be used
Similarity score: 0.565639641780277
--------

Chunk: When the Starter determines that the Race may be safely and 
fairly started , the Starter shall call out “Quick Start!” followed by the starting commands ( Rule 2 -306, 
“Starting Commands with Flags”  or Rule 2 -306.1, “Starting Commands with L ights” ). Once the 
announcement of “Quick Start” is made, t he Starter shall disregard any C rew signaling that it is not ready
Similarity score: 0.5589279422193736
--------

Chunk:  17 
2-306 Starting Commands with Flags  ................................ ................................ ................................ . 18 
2-306.1 Starting Commands with Lights  ................................ ............

In [113]:
'''
Connect to the LLM and generate a response to a query using the top 3 most similar chunks.

Parameters:
query - a string query
top_similarities - a list of tuples containing the most similar chunks and their similarity scores

Returns: a string response
'''
def connect_to_LLM(query, top_similarities):
    # Prompt gpt-3.5 to generate a response
    client = openai.Client()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Using the top 3 most similar chunks {top_similarities}, please generate a response to the query: {query}. If none of them are relevant, please say `I don't know`."
            }
        ])
    print(query)
    return response.choices[0].message.content

print(connect_to_LLM(query, top_similarity_scores))

How does a quick start work?
A Quick Start in rowing involves the Starter skipping the announcement of Crews and directly calling out "Quick Start!" followed by the starting commands. This allows for a faster and more efficient start to the race in case of weather or other conditions preventing the normal starting procedure.
